In [149]:
import numpy as np
import talib
import pandas as pd
import pathlib
from talib import abstract

In [150]:
PATH = r'./data.csv'
df = pd.read_csv(PATH, thousands=',')

In [151]:
df

,Date,Price,Open,High,Low,Vol.,Change %
0,"Dec 29, 2017",14392.6,14398.5,15109.8,13951.1,118.88K,-0.04%
1,"Dec 28, 2017",14398.7,15416.3,15505.5,13466.1,170.37K,-6.60%
2,"Dec 27, 2017",15416.6,15757.0,16514.6,14534.7,138.71K,-2.16%
3,"Dec 26, 2017",15756.6,13830.2,16094.7,13748.5,143.14K,13.90%
4,"Dec 25, 2017",13833.5,13790.0,14467.4,13010.7,107.48K,0.32%
...,...,...,...,...,...,...,...
2185,"Jan 05, 2012",6.9,5.6,7.2,5.6,182.33K,24.78%
2186,"Jan 04, 2012",5.6,4.9,5.7,4.8,131.17K,14.14%
2187,"Jan 03, 2012",4.9,5.2,5.3,4.7,125.17K,-6.51%
2188,"Jan 02, 2012",5.2,5.3,5.5,4.8,69.15K,-0.95%


### Splitting

Whole data set: 01.01.2012 - 29.12.2017 | 2190 (should be 2168)

- 02.01.2012 - 29.04.2012 | 120
- 30.04.2012 - 19.07.2016 (training) | 1539
- 20.07.2016 - 29.12.2017 (test) | 509

In [152]:
df = df.rename(columns={"Date": "date", 'Price': 'close', 'Open': 'open', "High": "high", "Low": "low", "Vol.": "volume", "Change %": "change"})

In [153]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index(df['date'])
df = df.sort_index()

In [154]:
df["change"] = df["change"].str.rstrip('%').astype('float')

In [155]:
df.volume = (df.volume.replace(r'[KMB]+$', '', regex=True).astype(float) * df.volume.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))

In [156]:
df["close"] = df["close"].astype('float')

In [157]:
df = df.astype({"close": "float64", "low": "float64", "high": "float64", "open": "float64"})

In [158]:
bins = [-100, -11, -9, -7, -5, -3, -1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1, 3 , 5, 7, 9, 11, 100]
df['binned'] = pd.cut(df['change'], bins)

In [159]:
df

,date,close,open,high,low,volume,change,binned
date,,,,,,,,
2012-01-01,2012-01-01,5.3,4.7,5.5,4.6,108510.0,11.65,"(11.0, 100.0]"
2012-01-02,2012-01-02,5.2,5.3,5.5,4.8,69150.0,-0.95,"(-1.0, -0.8]"
2012-01-03,2012-01-03,4.9,5.2,5.3,4.7,125170.0,-6.51,"(-7.0, -5.0]"
2012-01-04,2012-01-04,5.6,4.9,5.7,4.8,131170.0,14.14,"(11.0, 100.0]"
2012-01-05,2012-01-05,6.9,5.6,7.2,5.6,182330.0,24.78,"(11.0, 100.0]"
...,...,...,...,...,...,...,...,...
2017-12-25,2017-12-25,13833.5,13790.0,14467.4,13010.7,107480.0,0.32,"(0.2, 0.4]"
2017-12-26,2017-12-26,15756.6,13830.2,16094.7,13748.5,143140.0,13.90,"(11.0, 100.0]"
2017-12-27,2017-12-27,15416.6,15757.0,16514.6,14534.7,138710.0,-2.16,"(-3.0, -1.0]"


In [160]:
indicators = [
    {
        "name": "BBANDS"
    },
    {
        "name": "DEMA"
    },
    {
        "name": "EMA"
    },
    {
        "name": "HT_TRENDLINE"
    },
    {
        "name": "KAMA"
    },
    {
        "name": "MIDPOINT"
    },
    {
        "name": "MIDPRICE"
    },
    {
        "name": "SAR"
    },
    {
        "name": "SAREXT"
    },
    {
        "name": "SMA",
        "display_name": "SMA_3",
        "params": {
            "timeperiod": 3
        }
    },
    {
        "name": "SMA",
        "display_name": "SMA_5",
        "params": {
            "timeperiod": 5
        }
    },
    {
        "name": "SMA",
        "display_name": "SMA_10",
        "params": {
            "timeperiod": 10
        }
    },
    {
        "name": "SMA",
        "display_name": "SMA_20",
        "params": {
            "timeperiod": 20
        }
    },
    {
        "name": "T3"
    },
    {
        "name": "TEMA"
    },
    {
        "name": "TRIMA"
    },
    {
        "name": "WMA"
    },
    {
        "name": "ADX",
        "display_name": "ADX_14",
        "params": {
            "timeperiod": 14
        }
    },
    {
        "name": "ADX",
        "display_name": "ADX_20",
        "params": {
            "timeperiod": 20
        }
    },
    {
        "name": "ADXR"
    },
    {
        "name": "APO"
    },
    {
        "name": "AROONOSC"
    },
    {
        "name": "BOP"
    },
    {
        "name": "CCI",
        "display_name": "CCI_3",
        "params": {
            "timeperiod": 3
        }
    },
    {
        "name": "CCI",
        "display_name": "CCI_5",
        "params": {
            "timeperiod": 5
        }
    },
    {
        "name": "CCI",
        "display_name": "CCI_10",
        "params": {
            "timeperiod": 10
        }
    },
    {
        "name": "CCI",
        "display_name": "CCI_14",
        "params": {
            "timeperiod": 14
        }
    },
    {
        "name": "CMO"
    },
    {
        "name": "DX"
    },
    {
        "name": "BOP"
    },
    {
        "name": "MACD"
    },
    {
        "name": "MINUS_DI"
    },
    {
        "name": "MINUS_DM"
    },
    {
        "name": "MOM",
        "display_name": "MOM_1",
        "params": {
            "timeperiod": 1
        }
    },
    {
        "name": "MOM",
        "display_name": "MOM_3",
        "params": {
            "timeperiod": 3
        }
    },
    {
        "name": "MOM",
        "display_name": "MOM_5",
        "params": {
            "timeperiod": 5
        }
    },
    {
        "name": "MOM",
        "display_name": "MOM_10",
        "params": {
            "timeperiod": 10
        }
    },
    {
        "name": "PLUS_DI"
    },
    {
        "name": "PLUS_DM"
    },
    {
        "name": "PPO"
    },
    {
        "name": "ROC"
    },
    {
        "name": "ROCR"
    },
    {
        "name": "ROCR100"
    },
    {
        "name": "RSI",
        "display_name": "RSI_5",
        "params": {
            "timeperiod": 5
        }
    },
    {
        "name": "RSI",
        "display_name": "RSI_10",
        "params": {
            "timeperiod": 10
        }
    },
    {
        "name": "RSI",
        "display_name": "RSI_15",
        "params": {
            "timeperiod": 15
        }
    },
    {
        "name": "STOCH"
    },
    {
        "name": "STOCHF"
    },
    {
        "name": "TRIX"
    },
    {
        "name": "ULTOSC"
    },
    {
        "name": "WILLR"
    },
    {
        "name": "ATR"
    },
    {
        "name": "NATR"
    },
    {
        "name": "TRANGE"
    },
    {
        "name": "CDL2CROWS"
    },
    {
        "name": "CDL3BLACKCROWS"
    },
    {
        "name": "CDL3INSIDE"
    },
    {
        "name": "CDL3LINESTRIKE"
    },
    {
        "name": "CDL3OUTSIDE"
    },
    {
        "name": "CDL3STARSINSOUTH"
    },
    {
        "name": "CDL3WHITESOLDIERS"
    },
    {
        "name": "CDLABANDONEDBABY"
    },
    {
        "name": "CDLADVANCEBLOCK"
    },
    {
        "name": "CDLBELTHOLD"
    },
    {
        "name": "CDLBREAKAWAY"
    },
    {
        "name": "CDLCLOSINGMARUBOZU"
    },
    {
        "name": "CDLCONCEALBABYSWALL"
    },
    {
        "name": "CDLCOUNTERATTACK"
    },
    {
        "name": "CDLDARKCLOUDCOVER"
    },
    {
        "name": "CDLDOJI"
    },
    {
        "name": "CDLDOJISTAR"
    },
    {
        "name": "CDLDRAGONFLYDOJI"
    },
    {
        "name": "CDLENGULFING"
    },
    {
        "name": "CDLEVENINGDOJISTAR"
    },
    {
        "name": "CDLEVENINGSTAR"
    },
    {
        "name": "CDLGAPSIDESIDEWHITE"
    },
    {
        "name": "CDLGRAVESTONEDOJI"
    },
    {
        "name": "CDLHAMMER"
    },
    {
        "name": "CDLHANGINGMAN"
    },
    {
        "name": "CDLHARAMI"
    },
    {
        "name": "CDLHARAMICROSS"
    },
    {
        "name": "CDLHIGHWAVE"
    },
    {
        "name": "CDLHIKKAKE"
    },
    {
        "name": "CDLHIKKAKEMOD"
    },
    {
        "name": "CDLHOMINGPIGEON"
    },
    {
        "name": "CDLIDENTICAL3CROWS"
    },
    {
        "name": "CDLINNECK"
    },
    {
        "name": "CDLINVERTEDHAMMER"
    },
    {
        "name": "CDLKICKING"
    },
    {
        "name": "CDLKICKINGBYLENGTH"
    },
    {
        "name": "CDLLADDERBOTTOM"
    },
    {
        "name": "CDLLONGLEGGEDDOJI"
    },
    {
        "name": "CDLLONGLINE"
    },
    {
        "name": "CDLMARUBOZU"
    },
    {
        "name": "CDLMATCHINGLOW"
    },
    {
        "name": "CDLMATHOLD"
    },
    {
        "name": "CDLMORNINGDOJISTAR"
    },
    {
        "name": "CDLMORNINGSTAR"
    },
    {
        "name": "CDLONNECK"
    },
    {
        "name": "CDLPIERCING"
    },
    {
        "name": "CDLRICKSHAWMAN"
    },
    {
        "name": "CDLRISEFALL3METHODS"
    },
    {
        "name": "CDLSEPARATINGLINES"
    },
    {
        "name": "CDLSHOOTINGSTAR"
    },
    {
        "name": "CDLSHORTLINE"
    },
    {
        "name": "CDLSPINNINGTOP"
    },
    {
        "name": "CDLSTALLEDPATTERN"
    },
    {
        "name": "CDLSTICKSANDWICH"
    },
    {
        "name": "CDLTAKURI"
    },
    {
        "name": "CDLTASUKIGAP"
    },
    {
        "name": "CDLTHRUSTING"
    },
    {
        "name": "CDLTRISTAR"
    },
    {
        "name": "CDLUNIQUE3RIVER"
    },
    {
        "name": "CDLUPSIDEGAP2CROWS"
    },
    {
        "name": "CDLXSIDEGAP3METHODS"
    },
    {
        "name": "HT_DCPERIOD"
    },
    {
        "name": "HT_DCPHASE"
    },
    {
        "name": "HT_TRENDMODE"
    }
]

In [168]:
fn = abstract.Function("CDLINNECK")
fn

{'name': 'CDLINNECK', 'group': 'Pattern Recognition', 'display_name': 'In-Neck Pattern', 'function_flags': ['Output is a candlestick'], 'input_names': OrderedDict([('prices', ['open', 'high', 'low', 'close'])]), 'parameters': OrderedDict(), 'output_flags': OrderedDict([('integer', ['Line'])]), 'output_names': ['integer']}

In [169]:
fn = abstract.Function("RSI")
fn

{'name': 'RSI', 'group': 'Momentum Indicators', 'display_name': 'Relative Strength Index', 'function_flags': ['Function has an unstable period'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 14)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}

## Indicators

In [162]:
for indicator in indicators:
    name = indicator.get("name")
    params = indicator.get("params")
    display_name = indicator.get("display_name")
    
    fn = abstract.Function(name)
    
    if params == None:
        res = fn(df)
    else:
        res = fn(df, **params)
    
    name = display_name if display_name != None else name
    
    if isinstance(res, pd.core.series.Series):
        res = res.rename(name)
    elif isinstance(res, pd.core.frame.DataFrame):
        new_columns = {}
        for col in res:
            new_columns[col] = name + "_" + col
        res.rename(columns=new_columns, inplace=True)

    df =  pd.concat([df, res], axis=1)


In [164]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [171]:
df.describe()

,close,open,high,low,volume,change,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,DEMA,EMA,HT_TRENDLINE,KAMA,MIDPOINT,MIDPRICE,SAR,SAREXT,SMA_3,SMA_5,SMA_10,SMA_20,T3,TEMA,TRIMA,WMA,ADX_14,ADX_20,ADXR,APO,AROONOSC,BOP,CCI_3,CCI_5,CCI_10,CCI_14,CMO,DX,BOP,MACD_macd,MACD_macdsignal,MACD_macdhist,MINUS_DI,MINUS_DM,MOM_1,MOM_3,MOM_5,MOM_10,PLUS_DI,PLUS_DM,PPO,ROC,ROCR,ROCR100,RSI_5,RSI_10,RSI_15,STOCH_slowk,STOCH_slowd,STOCHF_fastk,STOCHF_fastd,TRIX,ULTOSC,WILLR,ATR,NATR,TRANGE,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLGAPSIDESIDEWHITE,CDLGRAVESTONEDOJI,CDLHAMMER,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLHIGHWAVE,CDLHIKKAKE,CDLHIKKAKEMOD,CDLHOMINGPIGEON,CDLIDENTICAL3CROWS,CDLINNECK,CDLINVERTEDHAMMER,CDLKICKING,CDLKICKINGBYLENGTH,CDLLADDERBOTTOM,CDLLONGLEGGEDDOJI,CDLLONGLINE,CDLMARUBOZU,CDLMATCHINGLOW,CDLMATHOLD,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLONNECK,CDLPIERCING,CDLRICKSHAWMAN,CDLRISEFALL3METHODS,CDLSEPARATINGLINES,CDLSHOOTINGSTAR,CDLSHORTLINE,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS,HT_DCPERIOD,HT_DCPHASE,HT_TRENDMODE
count,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2186.000000,2186.000000,2186.000000,2132.000000,2161.000000,2127.000000,2160.000000,2177.000000,2177.000000,2189.000000,2189.000000,2188.000000,2186.000000,2181.000000,2171.000000,2166.000000,2103.000000,2161.000000,2161.000000,2163.000000,2151.000000,2150.000000,2165.000000,2176.000000,2190.000000,2188.000000,2186.000000,2181.000000,2177.000000,2176.000000,2176.000000,2190.000000,2157.000000,2157.000000,2157.000000,2176.000000,2177.000000,2189.000000,2187.000000,2185.000000,2180.000000,2176.000000,2177.000000,2165.000000,2180.000000,2180.000000,2180.000000,2185.000000,2180.000000,2175.000000,2.182000e+03,2182.000000,2184.000000,2.184000e+03,2102.000000,2162.000000,2177.000000,2176.000000,2176.000000,2189.000000,2190.0,2190.0,2190.000000,2190.0,2190.0,2190.0,2190.000000,2190.0,2190.000000,2190.000000,2190.0,2190.000000,2190.0,2190.0,2190.0,2190.000000,2190.00000,2190.000000,2190.0,2190.0,2190.0,2190.0,2190.000000,2190.000000,2190.000000,2190.000000,2190.0,2190.000000,2190.000000,2190.000000,2190.0,2190.0,2190.0,2190.0,2190.0,2190.0,2190.0,2190.000000,2190.000000,2190.000000,2190.000000,2190.0,2190.0,2190.0,2190.0,2190.0,2190.000000,2190.0,2190.000000,2190.0,2190.000000,2190.000000,2190.000000,2190.0,2190.000000,2190.0,2190.0,2190.0,2190.0,2190.0,2190.0,2158.000000,2127.000000,2190.000000
mean,913.453881,906.881598,943.633379,869.993744,57980.694064,0.608178,978.248713,901.638747,825.028780,923.145350,829.647675,843.686610,893.964431,875.737575,864.403169,859.995213,122.832991,907.707541,901.638747,887.057437,853.531435,911.899071,953.561120,820.158928,856.278178,30.667763,26.245509,30.624990,53.347378,19.262080,0.078595,14.334261,19.983153,25.792665,30.251472,12.949944,30.714448,0.078595,46.101216,44.207742,1.893474,19.707006,166.002005,6.572545,20.206904,33.762059,67.744220,23.793216,237.657543,2.541106,5.647159,1.056472,105.647159,57.079715,56.669907,56.436042,5.867820e+01,58.689474,58.684604,5.870265e+01,0.359899,55.892303,-39.886721,63.143624,6.189315,73.860576,0.0,0.0,0.045662,0.0,0.0,0.0,0.136986,0.0,-1.735160,2.602740,0.0,3.881279,0.0,0.0,0.0,20.091324,0.00000,2.876712,0.0,0.0,0.0,0.0,1.689498,2.420091,-1.826484,0.045662,0.0,1.506849,0.045662,-0.045662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.589041,5.662100,1.415525,1.780822,0.0,0.0,0.0,0.0,0.0,14.520548,0.0,-0.273973,0.0,3.150685,2.465753,-0.273973,0.0,2.785388,0.0,0.0,0.0,0.0,0.0,0.0,23.253379,139.881984,0.835616
std,2104.922934,2085.197192,2194.919523,1965.082588,55408.184755,9.219659,2303.025809,2059

In [172]:
init_start = "2012-01-02"
init_end = "2012-04-29"

train_start = "2012-04-30"
train_end = "2016-07-19"

test_start = "2016-07-20"
test_end = "2017-12-29"

In [173]:
# create train test partition
init = df[init_start:init_end]
train = df[train_start:train_end]
test  = df[test_start:test_end]

print('Init Dataset:',init.shape)
print('Train Dataset:',train.shape)
print('Test Dataset:',test.shape)

Init Dataset: (119, 132)
Train Dataset: (1542, 132)
Test Dataset: (528, 132)


# Machine learning magic

In [174]:
from sklearn.tree import DecisionTreeClassifier